<a href="https://colab.research.google.com/github/smhall97/hallucinating_GANs/blob/main/DeepDreamMusic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# @title Imports

import torch
import torch.nn as nn
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import argparse
import os
import tqdm
import scipy.ndimage as nd
import os
import torchvision
import torchaudio
import IPython

In [ ]:
# @title Helper functions (mean, std)

mean = np.array([0.485, 0.456, 0.406])
std = np.array([0.229, 0.224, 0.225])

normalize = torchvision.transforms.Normalize(mean, std)
preprocess = torchvision.transforms.Compose([torchvision.transforms.ToTensor(), normalize])


def deprocess(image_np):
    print("Deproccess, incoming image: ", type(image_np), image_np.shape)
    image_np = image_np.transpose(1, 2, 0)
    image_np = image_np * std.reshape((1, 1, 3)) + mean.reshape((1, 1, 3))
    image_np = np.clip(image_np, 0.0, 255.0)
    return image_np

def clip(image_tensor):
    for c in range(3):
        m, s = mean[c], std[c]
        image_tensor[0, c] = torch.clamp(image_tensor[0, c], -m / s, (1 - m) / s)
    return image_tensor

# Load model

In [ ]:
# TODO: get params from Pablo for the path name

path_to_model = f'/content/{}_{}.pt Load from drive

model_state_dict = torch.load(path_to_model)
trained_model.load_state_dict(model_state_dict)

# Freeze parameters (we are not doing any model training here!)
for param in trained_model.parameters():
    param.requires_grad = False

# Get input image for dreaming



In [ ]:
# TODO: images for training

# Dreaming function (without octaves)

In [ ]:
# @title Dreaming function

def dream(image, model, iterations, lr, unit=None):
    """ Updates the image to maximize outputs for n iterations """
    
    if unit is None:
        print(f"Dreaming ... {image.shape}")
    else:
        print(f"Dream of {unit} {inet_classes[unit]} ... {image.shape}")

    image = preprocess(image)

    # Convert the numpy array image into a requires_grad=True Tensor that we will optimize
    image = torch.tensor(image, requires_grad=True, dtype=torch.float, device=DEVICE)
            
    # Optimization loop
    for i in range(iterations):
        model.zero_grad()
        out = model(image.unsqueeze(0))

        if unit is None:
            # Dreaming on a whole layer 
            gain = out.norm()
        else:
            # Dreaming on a specific output neuron...
            gain = out[0][unit]
            
        if i % 50 == 0:
            print(f"{i}: activation {gain.shape}: {gain}")
                        
        gain.backward()
        
        # Update image using gradients...
        #print(i, image.grad)
        # print("data: ", image.data)
        #avg_grad = np.abs(image.grad.data.cpu().numpy()).mean()
        with torch.no_grad():
            absgrad = torch.abs(image.grad)
            norm_lr = lr / absgrad.mean()
            image += norm_lr * image.grad
            image = clip(image)
        
        image.grad.zero_()

    return deprocess(image.detach().cpu().numpy())

### Dreaming hyperparameters and input image

In [ ]:
# Load image to be dreamed:
image = ''

# Hyper parameters for deep dreaming
dreaming_layer = -1 # output layer
dreaming_unit = int(1) # depends on genre of interest
learning_rate = 1e-02
iterations = 20 # number of times the deep dream loop should be run

### Run the dreaming function

In [ ]:
dreamed_image = dream(image, trained_model, iterations, learning_rate, unit=dreaming_unit)